# 1. Loading and Preprocessing of Data

## 1.1 Import Libraries

In [17]:
import pandas as pd
import random
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error
import pickle
from sklearn.preprocessing import LabelEncoder
from math import sqrt  # Import the sqrt function from the math module
from pmdarima import auto_arima
import matplotlib.pyplot as plt

## 1.2 Import Data

In [18]:
# Load qualitative data
df_qdm = pd.read_csv("qualitative_data_msba.csv")

# Load time series data
df_tsdm = pd.read_csv("time_series_data_msba.csv")

## 1.3 Cleaning the Data

In [19]:
# Imputing Missing values with 'None' and 'Not Present' categories
df_qdm['rv_lanes_stack_type'] = df_qdm['rv_lanes_stack_type'].fillna('None')
df_qdm['hi_flow_rv_lanes_stack_type'] = df_qdm['hi_flow_rv_lanes_stack_type'].fillna('None')
df_qdm = df_qdm.fillna('Not Present')

In [20]:
# Initialize a LabelEncoder
label_encoder = LabelEncoder()

In [21]:
# Encode categorical columns in 'categorical_vars'
categorical_vars = ['lottery', 'freal', 'bonfire_grill', 'pizza', 'cinnabon', 'ethanol_free', 'hi_flow_lanes', 'rv_lanes', 'cat_scales', 'rv_dumps', 'propane', 'traditional_forecourt_layout', 'traditional_forecourt_stack_type', 'rv_lanes_layout', 'rv_lanes_stack_type', 'hi_flow_lanes_layout', 'hi_flow_rv_lanes_layout']
for column in categorical_vars:
    if column in df_qdm.columns:
        df_qdm[column] = label_encoder.fit_transform(df_qdm[column])

Missing values in specific columns are imputed with 'None' and 'Not Present' categories. Categorical columns are encoded using LabelEncoder, transforming categorical values into numerical representations.

## 1.4 Merging Datasets

In [22]:
# Merge data on 'site_id_msba'
merged_df = pd.merge(df_qdm, df_tsdm, on='site_id_msba')

## 1.5 Cleaning unnecessary Columns

In [23]:
#removing the un-named column from the data
merged_df.drop('Unnamed: 0_x', axis=1, inplace=True)
merged_df.drop('Unnamed: 0_y', axis=1, inplace=True)

#checking to make sure it was removed correctly
merged_df.head()

,open_year,square_feet,front_door_count,years_since_last_project,parking_spaces,lottery,freal,bonfire_grill,pizza,cinnabon,...,capital_projects.soft_opening_date,calendar.calendar_day_date,calendar.fiscal_week_id_for_year,calendar.day_of_week,calendar_information.holiday,calendar_information.type_of_day,daily_yoy_ndt.total_inside_sales,daily_yoy_ndt.total_food_service,diesel_y,unleaded
0,2021,5046,2,2,38,1,1,1,0,0,...,2021-01-12,2021-01-17,3,Sunday,NONE,WEEKEND,1792.1260,528.8185,870.0160,1490.3980
1,2021,5046,2,2,38,1,1,1,0,0,...,2021-01-12,2021-01-20,3,Wednesday,NONE,WEEKDAY,2297.4595,793.7300,1582.0420,1750.7525
2,2021,5046,2,2,38,1,1,1,0,0,...,2021-01-12,2021-01-21,3,Thursday,NONE,WEEKDAY,2381.5120,829.0730,1506.6660,1727.6210
3,2021,5046,2,2,38,1,1,1,0,0,...,2021-01-12,2021-01-24,4,Sunday,NONE,WEEKEND,1396.4370,456.1830,510.9370,1184.8060
4,2021,5046,2,2,38,1,1,1,0,0,...,2021-01-12,2021-01-29,5,Friday,NONE,WEEKDAY,2568.2370,782.6245,1604.0325,1987.2720


## 1.6 Checking for Unique Store Sites

In [24]:
#checking how many unique sites there are
num_distinct_values = merged_df['site_id_msba'].nunique()
print('There are', num_distinct_values, 'unique sites.')

There are 37 unique sites.


In [25]:
#converting the 'calendar.calendar_day_date' column to datetime
merged_df['calendar.calendar_day_date'] = pd.to_datetime(merged_df['calendar.calendar_day_date'])

#creating a new column 'day_of_year' with the day of the year
merged_df['day_of_year'] = merged_df['calendar.calendar_day_date'].dt.dayofyear

merged_df.head()

,open_year,square_feet,front_door_count,years_since_last_project,parking_spaces,lottery,freal,bonfire_grill,pizza,cinnabon,...,calendar.calendar_day_date,calendar.fiscal_week_id_for_year,calendar.day_of_week,calendar_information.holiday,calendar_information.type_of_day,daily_yoy_ndt.total_inside_sales,daily_yoy_ndt.total_food_service,diesel_y,unleaded,day_of_year
0,2021,5046,2,2,38,1,1,1,0,0,...,2021-01-17,3,Sunday,NONE,WEEKEND,1792.1260,528.8185,870.0160,1490.3980,17
1,2021,5046,2,2,38,1,1,1,0,0,...,2021-01-20,3,Wednesday,NONE,WEEKDAY,2297.4595,793.7300,1582.0420,1750.7525,20
2,2021,5046,2,2,38,1,1,1,0,0,...,2021-01-21,3,Thursday,NONE,WEEKDAY,2381.5120,829.0730,1506.6660,1727.6210,21
3,2021,5046,2,2,38,1,1,1,0,0,...,2021-01-24,4,Sunday,NONE,WEEKEND,1396.4370,456.1830,510.9370,1184.8060,24
4,2021,5046,2,2,38,1,1,1,0,0,...,2021-01-29,5,Friday,NONE,WEEKDAY,2568.2370,782.6245,1604.0325,1987.2720,29


# 2. Splitting Data into Training and Testing Sets

In [26]:
random.seed(42)  # Set a random seed for reproducibility
unique_sites = merged_df['site_id_msba'].unique()
random.shuffle(unique_sites)
training_sites = unique_sites[:30]
testing_sites = unique_sites[30:]

print("Selected Sites for Training Data:", training_sites)
print("Selected Sites for Testing Data:", testing_sites)

Selected Sites for Training Data: [22260 23135 22680 22540 23345 23485 23555 22120 23415 23730 24150 23835
 23905 22575 22645 22925 23660 22820 24535 22330 23450 21560 24255 22715
 22890 22015 24220 23765 23240 22085]
Selected Sites for Testing Data: [23380 22505 22750 22785 22855 21980 22400]


# 3. Model Training and Evaluation

In [29]:
target_variables = ['diesel_y', 'unleaded', 'daily_yoy_ndt.total_inside_sales', 'daily_yoy_ndt.total_food_service']

for target_variable in target_variables:
    train_data = merged_df[merged_df['site_id_msba'].isin(training_sites)][['capital_projects.soft_opening_date', target_variable]]
    test_data = merged_df[merged_df['site_id_msba'].isin(testing_sites)][['capital_projects.soft_opening_date', target_variable]]

    train_data['capital_projects.soft_opening_date'] = pd.to_datetime(train_data['capital_projects.soft_opening_date'])
    test_data['capital_projects.soft_opening_date'] = pd.to_datetime(test_data['capital_projects.soft_opening_date'])

    train_data.set_index('capital_projects.soft_opening_date', inplace=True)
    test_data.set_index('capital_projects.soft_opening_date', inplace=True)

    model = auto_arima(train_data[target_variable], seasonal=True, m=12, stepwise=True, suppress_warnings=True, error_action="ignore", max_order=None, trace=True)
    best_order = model.order
    best_seasonal_order = model.seasonal_order

    print(f"Best SARIMA Order for {target_variable}: {best_order}")
    print(f"Best SARIMA Seasonal Order for {target_variable}: {best_seasonal_order}")

    sarima_model = SARIMAX(train_data[target_variable], order=best_order, seasonal_order=best_seasonal_order)
    sarima_model_fit = sarima_model.fit(disp=False)

    predictions = sarima_model_fit.get_forecast(steps=len(test_data))
    predicted_values = predictions.predicted_mean

    rmse = sqrt(mean_squared_error(test_data[target_variable], predicted_values))
    print(f'RMSE for {target_variable}: {rmse}')

Performing stepwise search to minimize aic
 ARIMA(2,1,2)(1,0,1)[12] intercept   : AIC=176046.926, Time=5.23 sec
 ARIMA(0,1,0)(0,0,0)[12] intercept   : AIC=180701.693, Time=0.11 sec
 ARIMA(1,1,0)(1,0,0)[12] intercept   : AIC=178280.779, Time=1.09 sec
 ARIMA(0,1,1)(0,0,1)[12] intercept   : AIC=176086.907, Time=4.46 sec
 ARIMA(0,1,0)(0,0,0)[12]             : AIC=180699.694, Time=0.07 sec
 ARIMA(2,1,2)(0,0,1)[12] intercept   : AIC=176044.927, Time=4.37 sec
 ARIMA(2,1,2)(0,0,0)[12] intercept   : AIC=176042.958, Time=1.11 sec
 ARIMA(2,1,2)(1,0,0)[12] intercept   : AIC=176044.928, Time=3.77 sec
 ARIMA(1,1,2)(0,0,0)[12] intercept   : AIC=175989.867, Time=6.35 sec
 ARIMA(1,1,2)(1,0,0)[12] intercept   : AIC=176026.358, Time=16.96 sec
 ARIMA(1,1,2)(0,0,1)[12] intercept   : AIC=176087.910, Time=3.85 sec
 ARIMA(1,1,2)(1,0,1)[12] intercept   : AIC=176089.922, Time=4.36 sec
 ARIMA(0,1,2)(0,0,0)[12] intercept   : AIC=176036.714, Time=2.44 sec
 ARIMA(1,1,1)(0,0,0)[12] intercept   : AIC=176036.131, Time

/Users/darylebilog/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/darylebilog/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/darylebilog/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/darylebilog/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecas

RMSE for diesel_y: 4083.4580557889117
           diesel_y
2021-08-19      NaN
2021-08-20      NaN
2021-08-21      NaN
2021-08-22      NaN
2021-08-23      NaN
2021-08-24      NaN
2021-08-25      NaN
2021-08-26      NaN
2021-08-27      NaN
2021-08-28      NaN
2021-08-29      NaN
2021-08-30      NaN
2021-08-31      NaN
2021-09-01      NaN
2021-09-02      NaN
2021-09-03      NaN
2021-09-04      NaN
2021-09-05      NaN
2021-09-06      NaN
2021-09-07      NaN
2021-09-08      NaN
2021-09-09      NaN
2021-09-10      NaN
2021-09-11      NaN
2021-09-12      NaN
2021-09-13      NaN
2021-09-14      NaN
2021-09-15      NaN
2021-09-16      NaN
Performing stepwise search to minimize aic
 ARIMA(2,1,2)(1,0,1)[12] intercept   : AIC=168045.966, Time=10.03 sec
 ARIMA(0,1,0)(0,0,0)[12] intercept   : AIC=171934.076, Time=0.12 sec
 ARIMA(1,1,0)(1,0,0)[12] intercept   : AIC=169914.158, Time=1.30 sec
 ARIMA(0,1,1)(0,0,1)[12] intercept   : AIC=168122.068, Time=6.45 sec
 ARIMA(0,1,0)(0,0,0)[12]             : AIC

/Users/darylebilog/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/darylebilog/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/darylebilog/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/darylebilog/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecas

RMSE for unleaded: 929.2464467226487
           unleaded
2021-08-19      NaN
2021-08-20      NaN
2021-08-21      NaN
2021-08-22      NaN
2021-08-23      NaN
2021-08-24      NaN
2021-08-25      NaN
2021-08-26      NaN
2021-08-27      NaN
2021-08-28      NaN
2021-08-29      NaN
2021-08-30      NaN
2021-08-31      NaN
2021-09-01      NaN
2021-09-02      NaN
2021-09-03      NaN
2021-09-04      NaN
2021-09-05      NaN
2021-09-06      NaN
2021-09-07      NaN
2021-09-08      NaN
2021-09-09      NaN
2021-09-10      NaN
2021-09-11      NaN
2021-09-12      NaN
2021-09-13      NaN
2021-09-14      NaN
2021-09-15      NaN
2021-09-16      NaN
Performing stepwise search to minimize aic
 ARIMA(2,1,2)(1,0,1)[12] intercept   : AIC=173396.820, Time=8.11 sec
 ARIMA(0,1,0)(0,0,0)[12] intercept   : AIC=178648.325, Time=0.13 sec
 ARIMA(1,1,0)(1,0,0)[12] intercept   : AIC=176210.380, Time=1.35 sec
 ARIMA(0,1,1)(0,0,1)[12] intercept   : AIC=173481.897, Time=1.78 sec
 ARIMA(0,1,0)(0,0,0)[12]             : AIC=1

/Users/darylebilog/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/darylebilog/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/darylebilog/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/darylebilog/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecas

RMSE for daily_yoy_ndt.total_inside_sales: 1230.6294569830532
           daily_yoy_ndt.total_inside_sales
2021-08-19                              NaN
2021-08-20                              NaN
2021-08-21                              NaN
2021-08-22                              NaN
2021-08-23                              NaN
2021-08-24                              NaN
2021-08-25                              NaN
2021-08-26                              NaN
2021-08-27                              NaN
2021-08-28                              NaN
2021-08-29                              NaN
2021-08-30                              NaN
2021-08-31                              NaN
2021-09-01                              NaN
2021-09-02                              NaN
2021-09-03                              NaN
2021-09-04                              NaN
2021-09-05                              NaN
2021-09-06                              NaN
2021-09-07                              NaN
2021-09-08    

/Users/darylebilog/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/darylebilog/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/darylebilog/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/darylebilog/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecas

RMSE for daily_yoy_ndt.total_food_service: 425.13391274636496
           daily_yoy_ndt.total_food_service
2021-08-19                              NaN
2021-08-20                              NaN
2021-08-21                              NaN
2021-08-22                              NaN
2021-08-23                              NaN
2021-08-24                              NaN
2021-08-25                              NaN
2021-08-26                              NaN
2021-08-27                              NaN
2021-08-28                              NaN
2021-08-29                              NaN
2021-08-30                              NaN
2021-08-31                              NaN
2021-09-01                              NaN
2021-09-02                              NaN
2021-09-03                              NaN
2021-09-04                              NaN
2021-09-05                              NaN
2021-09-06                              NaN
2021-09-07                              NaN
2021-09-08    

/Users/darylebilog/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/Users/darylebilog/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
/Users/darylebilog/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/Users/darylebilog/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model with

For each target variable, the training and testing data are filtered. The 'capital_projects.soft_opening_date' column is converted to datetime format, and this column is set as the index for both training and testing datasets.

A grid search is performed using auto_arima to find the best SARIMA parameters for the training data. The best order and seasonal order obtained from the grid search are used to fit a SARIMA model. Predictions are made on the testing data, and RMSE (Root Mean Square Error) is calculated to evaluate the model's performance for each target variable.

This model gave us the following RMSE for each Target Variable:

- RMSE for diesel_y: 4083.4580557889117
- RMSE for unleaded: 929.2464467226487
- RMSE for daily_yoy_ndt.total_inside_sales: 1230.6294569830532
- RMSE for daily_yoy_ndt.total_food_service: 425.13391274636496

In [33]:
from sklearn.metrics import mean_squared_error, r2_score
from math import sqrt

target_variables = ['diesel_y', 'unleaded', 'daily_yoy_ndt.total_inside_sales', 'daily_yoy_ndt.total_food_service']

for target_variable in target_variables:
    train_data = merged_df[merged_df['site_id_msba'].isin(training_sites)][['capital_projects.soft_opening_date', target_variable]]
    test_data = merged_df[merged_df['site_id_msba'].isin(testing_sites)][['capital_projects.soft_opening_date', target_variable]]

    train_data['capital_projects.soft_opening_date'] = pd.to_datetime(train_data['capital_projects.soft_opening_date'])
    test_data['capital_projects.soft_opening_date'] = pd.to_datetime(test_data['capital_projects.soft_opening_date'])

    train_data.set_index('capital_projects.soft_opening_date', inplace=True)
    test_data.set_index('capital_projects.soft_opening_date', inplace=True)

    model = auto_arima(train_data[target_variable], seasonal=True, m=12, stepwise=True, suppress_warnings=True, error_action="ignore", max_order=None, trace=True)
    best_order = model.order
    best_seasonal_order = model.seasonal_order

    print(f"Best SARIMA Order for {target_variable}: {best_order}")
    print(f"Best SARIMA Seasonal Order for {target_variable}: {best_seasonal_order}")

    sarima_model = SARIMAX(train_data[target_variable], order=best_order, seasonal_order=best_seasonal_order)
    sarima_model_fit = sarima_model.fit(disp=False)

    predictions = sarima_model_fit.get_forecast(steps=len(test_data))
    predicted_values = predictions.predicted_mean

    mse = mean_squared_error(test_data[target_variable], predicted_values)
    rmse = sqrt(mse)
    r_squared = r2_score(test_data[target_variable], predicted_values)
    
    print(f'RMSE for {target_variable}: {rmse}')
    print(f'MSE for {target_variable}: {mse}')
    print(f'R-squared for {target_variable}: {r_squared}')

Performing stepwise search to minimize aic
 ARIMA(2,1,2)(1,0,1)[12] intercept   : AIC=176046.926, Time=6.04 sec
 ARIMA(0,1,0)(0,0,0)[12] intercept   : AIC=180701.693, Time=0.12 sec
 ARIMA(1,1,0)(1,0,0)[12] intercept   : AIC=178280.779, Time=1.15 sec
 ARIMA(0,1,1)(0,0,1)[12] intercept   : AIC=176086.907, Time=4.59 sec
 ARIMA(0,1,0)(0,0,0)[12]             : AIC=180699.694, Time=0.07 sec
 ARIMA(2,1,2)(0,0,1)[12] intercept   : AIC=176044.927, Time=4.40 sec
 ARIMA(2,1,2)(0,0,0)[12] intercept   : AIC=176042.958, Time=1.14 sec
 ARIMA(2,1,2)(1,0,0)[12] intercept   : AIC=176044.928, Time=3.86 sec
 ARIMA(1,1,2)(0,0,0)[12] intercept   : AIC=175989.867, Time=6.49 sec
 ARIMA(1,1,2)(1,0,0)[12] intercept   : AIC=176026.358, Time=17.21 sec
 ARIMA(1,1,2)(0,0,1)[12] intercept   : AIC=176087.910, Time=3.91 sec
 ARIMA(1,1,2)(1,0,1)[12] intercept   : AIC=176089.922, Time=4.46 sec
 ARIMA(0,1,2)(0,0,0)[12] intercept   : AIC=176036.714, Time=2.41 sec
 ARIMA(1,1,1)(0,0,0)[12] intercept   : AIC=176036.131, Time

/Users/darylebilog/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/darylebilog/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/darylebilog/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/darylebilog/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecas

RMSE for diesel_y: 4083.4580557889117
MSE for diesel_y: 16674629.693387358
R-squared for diesel_y: -0.11226810759213102
Performing stepwise search to minimize aic
 ARIMA(2,1,2)(1,0,1)[12] intercept   : AIC=168045.966, Time=9.43 sec
 ARIMA(0,1,0)(0,0,0)[12] intercept   : AIC=171934.076, Time=0.12 sec
 ARIMA(1,1,0)(1,0,0)[12] intercept   : AIC=169914.158, Time=1.22 sec
 ARIMA(0,1,1)(0,0,1)[12] intercept   : AIC=168122.068, Time=6.00 sec
 ARIMA(0,1,0)(0,0,0)[12]             : AIC=171932.076, Time=0.07 sec
 ARIMA(2,1,2)(0,0,1)[12] intercept   : AIC=168043.997, Time=8.95 sec
 ARIMA(2,1,2)(0,0,0)[12] intercept   : AIC=168042.210, Time=2.49 sec
 ARIMA(2,1,2)(1,0,0)[12] intercept   : AIC=168043.995, Time=8.34 sec
 ARIMA(1,1,2)(0,0,0)[12] intercept   : AIC=168040.269, Time=1.40 sec
 ARIMA(1,1,2)(1,0,0)[12] intercept   : AIC=168042.060, Time=4.40 sec
 ARIMA(1,1,2)(0,0,1)[12] intercept   : AIC=168042.063, Time=4.55 sec
 ARIMA(1,1,2)(1,0,1)[12] intercept   : AIC=168044.048, Time=6.46 sec
 ARIMA(0,

/Users/darylebilog/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/darylebilog/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/darylebilog/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/darylebilog/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecas

RMSE for unleaded: 929.2464467226487
MSE for unleaded: 863498.9587466683
R-squared for unleaded: -0.7259441791332135
Performing stepwise search to minimize aic
 ARIMA(2,1,2)(1,0,1)[12] intercept   : AIC=173396.820, Time=7.24 sec
 ARIMA(0,1,0)(0,0,0)[12] intercept   : AIC=178648.325, Time=0.11 sec
 ARIMA(1,1,0)(1,0,0)[12] intercept   : AIC=176210.380, Time=1.24 sec
 ARIMA(0,1,1)(0,0,1)[12] intercept   : AIC=173481.897, Time=1.59 sec
 ARIMA(0,1,0)(0,0,0)[12]             : AIC=178646.326, Time=0.07 sec
 ARIMA(2,1,2)(0,0,1)[12] intercept   : AIC=173394.922, Time=7.59 sec
 ARIMA(2,1,2)(0,0,0)[12] intercept   : AIC=173398.004, Time=1.55 sec
 ARIMA(2,1,2)(0,0,2)[12] intercept   : AIC=173396.543, Time=18.55 sec
 ARIMA(2,1,2)(1,0,0)[12] intercept   : AIC=173394.871, Time=5.82 sec
 ARIMA(2,1,2)(2,0,0)[12] intercept   : AIC=173396.647, Time=18.43 sec
 ARIMA(2,1,2)(2,0,1)[12] intercept   : AIC=173398.659, Time=23.52 sec
 ARIMA(1,1,2)(1,0,0)[12] intercept   : AIC=173430.038, Time=4.26 sec
 ARIMA(2,

/Users/darylebilog/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/darylebilog/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/darylebilog/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/darylebilog/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecas

RMSE for daily_yoy_ndt.total_inside_sales: 1230.6294569830532
MSE for daily_yoy_ndt.total_inside_sales: 1514448.8603944043
R-squared for daily_yoy_ndt.total_inside_sales: -0.23403370737677798
Performing stepwise search to minimize aic
 ARIMA(2,1,2)(1,0,1)[12] intercept   : AIC=149932.812, Time=20.06 sec
 ARIMA(0,1,0)(0,0,0)[12] intercept   : AIC=155941.635, Time=0.13 sec
 ARIMA(1,1,0)(1,0,0)[12] intercept   : AIC=153386.052, Time=1.29 sec
 ARIMA(0,1,1)(0,0,1)[12] intercept   : AIC=150008.008, Time=4.81 sec
 ARIMA(0,1,0)(0,0,0)[12]             : AIC=155939.636, Time=0.07 sec
 ARIMA(2,1,2)(0,0,1)[12] intercept   : AIC=149930.828, Time=16.71 sec
 ARIMA(2,1,2)(0,0,0)[12] intercept   : AIC=149931.353, Time=4.33 sec
 ARIMA(2,1,2)(0,0,2)[12] intercept   : AIC=149932.181, Time=47.57 sec
 ARIMA(2,1,2)(1,0,0)[12] intercept   : AIC=149930.791, Time=15.34 sec
 ARIMA(2,1,2)(2,0,0)[12] intercept   : AIC=149932.302, Time=51.71 sec
 ARIMA(2,1,2)(2,0,1)[12] intercept   : AIC=149934.899, Time=61.35 sec


/Users/darylebilog/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/darylebilog/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/darylebilog/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/darylebilog/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecas

RMSE for daily_yoy_ndt.total_food_service: 425.13391274636496
MSE for daily_yoy_ndt.total_food_service: 180738.84376703383
R-squared for daily_yoy_ndt.total_food_service: -0.2454318323145137


/Users/darylebilog/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/Users/darylebilog/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
from math import sqrt

# Specify the best SARIMA order and seasonal order
best_order = (4, 1, 2)
best_seasonal_order = (1, 0, 0, 12)
target_variable = 'daily_yoy_ndt.total_food_service'

# Filter data for the specific target variable
train_data = merged_df[merged_df['site_id_msba'].isin(training_sites)][['capital_projects.soft_opening_date', target_variable]]
test_data = merged_df[merged_df['site_id_msba'].isin(testing_sites)][['capital_projects.soft_opening_date', target_variable]]

# Convert date columns to datetime format and set index
train_data['capital_projects.soft_opening_date'] = pd.to_datetime(train_data['capital_projects.soft_opening_date'])
test_data['capital_projects.soft_opening_date'] = pd.to_datetime(test_data['capital_projects.soft_opening_date'])
train_data.set_index('capital_projects.soft_opening_date', inplace=True)
test_data.set_index('capital_projects.soft_opening_date', inplace=True)

# Fit SARIMA model with the specified order and seasonal order
sarima_model = SARIMAX(train_data[target_variable], order=best_order, seasonal_order=best_seasonal_order)
sarima_model_fit = sarima_model.fit(disp=False)

# Make predictions on the test data
predictions = sarima_model_fit.get_forecast(steps=len(test_data))
predicted_values = predictions.predicted_mean

# Calculate metrics
mse = mean_squared_error(test_data[target_variable], predicted_values)
rmse = sqrt(mse)
r_squared = r2_score(test_data[target_variable], predicted_values)

# Print metrics
print(f'RMSE for {target_variable}: {rmse}')
print(f'MSE for {target_variable}: {mse}')
print(f'R-squared for {target_variable}: {r_squared}')